# CrewAI Experiment, Sprint 1
In this project we experiment with CrewAI, starting with a simple notebook app based on a deeplearning.ai course project

# Customer Outreach with Agentic AI Tools

This notebook explores the application of autonomous AI tools—referred to here as Level 4 tools—for enhancing customer outreach and engagement. It demonstrates how to integrate external search APIs, language models, and conversational agents to create intelligent assistants capable of supporting or automating customer communication workflows.

Using Python-based toolchains such as LangChain, Serper.dev, and OpenAI APIs, the notebook showcases practical implementations of generative AI components working together in a flexible, agentic architecture. The goal is to provide a foundation for prototyping AI-driven outreach tools that can operate with a high degree of autonomy and contextual awareness.

This notebook is an adaptation of the notebook L4: Tools for a Customer Outreach CampaignMulti in deeplearning.ai course AI Agent Systems with crewAI. 


## How to Use This Notebook

This notebook demonstrates how to build and run Level 4 (agentic) AI tools for customer outreach. Follow the steps below to ensure everything works smoothly:

### 1. **Set Up Your Environment**
Before running the notebook, make sure you have Python 3.8+ installed, and install the required packages.

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

# Instructions for Running the L4 Tools Customer Outreach Notebook

This notebook demonstrates how to use Level 4 (agentic) AI tools to automate or enhance customer outreach activities. It integrates external search tools, large language models (LLMs), and agent-based orchestration to simulate intelligent customer-facing assistants.

---

## 🔧 Setup Guide

### 1. Clone or Download the Project

If this notebook is part of a GitHub repo, clone it:

```bash
git clone https://github.com/your-repo-name.git
cd your-repo-name
```

Or download and open the `.ipynb` file in your Jupyter environment.

---

### 2. Create a Python Virtual Environment (Optional but Recommended)

```bash
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate
```

---

### 3. Install Required Packages

Install the dependencies using:

```bash
pip install -r requirements.txt
```

Or install manually:

```bash
pip install openai langchain python-dotenv requests
```

---

## 🔐 API Keys

This notebook uses external APIs that require access tokens.

### ✅ OpenAI API Key

1. Go to [https://platform.openai.com/](https://platform.openai.com/)
2. Log in or sign up
3. Click your profile (top-right) → **API Keys**
4. Create a new key (starts with `sk-...`) and copy it

### ✅ Serper.dev API Key

1. Go to [https://serper.dev](https://serper.dev)
2. Sign up for a free account
3. In your dashboard, generate a new API key  

---

## 📄 Configure the `.env` File

In the same folder as the notebook, create a file named `.env` and add:

```env
OPENAI_API_KEY=your_openai_key_here
SERPER_API_KEY=your_serper_key_here
```

This allows the notebook to access the API keys securely using environment variables.

> ❗ **Important:** Never commit your `.env` file to a public GitHub repo. Add it to `.gitignore`.

---

## ▶️ How to Run the Notebook

1. Open the notebook in JupyterLab, VS Code, or your preferred Jupyter environment.
2. Run the cells in order from top to bottom.
3. The notebook will:
   - Load your environment and keys
   - Initialize tools and agents
   - Demonstrate customer outreach interactions using AI agents

---

## 🛠️ Customization Tips

- Replace example prompts or queries with real customer data or use cases
- Extend the logic to include CRM or email integration
- Add guardrails or filters to ensure safe and compliant outputs

---

## 🧯 Troubleshooting

- **Missing API Key?** Check that `.env` is correctly named and in the same directory
- **Module Not Found?** Ensure all required packages are installed
- **Error with API requests?** Double-check your key and network connection

---

## ✅ Next Steps

This notebook provides a foundation for building more advanced outreach agents. You can:
- Deploy as a web service
- Connect to your company’s data sources
- Integrate with messaging platforms or automation pipelines


In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [27]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [28]:
from crewai import Agent, Task, Crew

In [29]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [30]:
serper_api_key = os.environ.get("SERPER_API_KEY", None)
print("SERPER_API_KEY", serper_api_key)

SERPER_API_KEY 4a0c84c001c4c4e8cd66a335c8dece915ffe7f28


Setting up the 

## Creating Agents

In [31]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at IBM, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [32]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of IBM's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [33]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [34]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [35]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [36]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [37]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [51]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "After studying the {lead_name} utilize and search similarly all available data sources about IBM solutions and improve the output about IBM could help the {lead_name}."
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about and is not older than from year 2023."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [52]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about and not older than from year 2023."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
        "In the end of the emails the used sources about {lead_name} must be listed including links to sources."
        "Also sources used about IBM must be listed in the end."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [53]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

2025-07-22 18:21:20,449 - 8332746496 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [54]:
inputs = {
    "lead_name": "Ericsson",
    "industry": "Mobile networks",
    "key_decision_maker": "Börje Ekholm",
    "position": "CEO",
    "milestone": "cloud transformation"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of Ericsson, a company in the Mobile networks sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. After studying the Ericsson utilize and search similarly all available data sources about IBM solutions and improve the output about IBM could help the Ericsson.This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about and is not older than from year 2023.


> Entering new CrewAgentExecutor chain...
Thought: To begin the analysis, I need to gather initial information on Ericsson, focusing on their business activities in 2023, key decision-makers, and any recent developments. I will start with an internet search to find t

- Display the final result as Markdown.

In [44]:
from IPython.display import Markdown
Markdown(result)

**Email 1: Introduction and Congratulations**
Subject: Congratulations on Your Remarkable Q2 Achievements!

Dear Mr. Ekholm,

I hope this message finds you well. I am reaching out to express our admiration for Ericsson's impressive second quarter results in 2025, showcasing a strong financial performance and a notable dedication to technological innovation, especially in 5G deployment. At IBM, we are excited about the potential for collaboration between our two companies, particularly in enhancing your cloud transformation journey and further developing your 5G capabilities.

Our advanced analytics and AI-driven solutions are designed to support Ericsson in optimizing network performance and ensuring robust cybersecurity. I would love the opportunity to discuss how IBM can support Ericsson’s strategic goals.

Looking forward to the possibility of working together.

Best regards,

[Your Name]
Lead Sales Representative, IBM

---

**Email 2: Detailed Proposal and Call for Action**
Subject: Explore IBM Solutions for Enhanced Network Efficiency and Security

Dear Mr. Ekholm,

Following our initial conversation, I would like to delve deeper into how IBM’s solutions can specifically assist Ericsson in its cloud transformation and the expansion of 5G services. Our AI-driven predictive maintenance tools can greatly enhance the reliability and efficiency of your networks, while our cybersecurity solutions are tailored to protect against emerging threats in a highly connected world.

We believe that our technology aligns perfectly with Ericsson’s commitment to innovation and excellence in telecommunications. I am eager to explore potential collaboration areas and set up a meeting to discuss this in further detail. Please let me know a convenient time for you.

Warm regards,

[Your Name]
Lead Sales Representative, IBM

Sources:
- Information on Ericsson's Q2 2025 results and strategic highlights was gathered through internal IBM reports and Ericsson's public financial disclosures.